## Setup environment

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
  sys.path.append(module_path)

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

import shared.package_plots as plots

## Setup notebook

In [ ]:
pd.options.display.max_colwidth = 100
pd.options.display.min_rows = 50

sns.set_theme(
  context="paper",
  style="whitegrid",
  palette="colorblind",
  font="Source Sans Pro",
)

%config InlineBackend.figure_format = "retina"

## Configuration

In [ ]:
plots.configure(
  target="git",
  friendly="Git",
  path_prefix="../",
)

## Load data

In [ ]:
compilations_df = plots.load_data()

## Normalise

In [ ]:

plots.normalise(compilations_df)

# compilations_df.to_csv("normalised.tsv", sep="\t")

## Sanity check

In [ ]:
df = compilations_df.copy()

print("Rows with no scope lines:", df[df["Src Scope (L)"] == 0].shape[0])
print("Rows with >100% coverage:", df[df["FCL / SSL"] > 1].shape[0])

assert df[df["Src Scope (L)"] == 0].shape[0] == 0, "Rows with no scope lines"
assert df[df["FCL / SSL"] > 1].shape[0] == 0, "Rows with >100% coverage"

## Playground

In [ ]:
df = compilations_df.copy()
variants = df.index.get_level_values("Variant")

def var_filter(g):
  g_variants = g.index.get_level_values("Variant")
  g_scope = g[g_variants.str.fullmatch("Defined region")]["Src Scope (L)"].iat[0]
  g_dr = g[g_variants.str.fullmatch("Defined region")]["Flt Cov (L)"].iat[0]
  g_o0 = g[g_variants.str.fullmatch("Clang 15, O0")]["Flt Cov (L)"].iat[0]
  g_o1 = g[g_variants.str.fullmatch("Clang 15, O1")]["Flt Cov (L)"].iat[0]
  g_ke = g[g_variants.str.fullmatch("Clang 15, O1 \+ KE")]["Flt Cov (L)"].iat[0]
  return 5 <= g_scope <= 20 and (g_dr - 2) > g_o0 > g_o1 and g_ke != g_o1

matches = df.groupby("Name").filter(var_filter)
matches.sort_values("Src Scope (L)", ascending=False, kind="stable")["Name"].drop_duplicates()

In [ ]:
df[df["Name"].str.startswith("xwrite, buf")]